In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [54]:
# dataset = pd.read_csv('C:/Users/mskim3/Desktop/식단인식_알고리즘/data/dataset.csv',encoding='cp949').iloc[:,1:]
# dataset.to_csv('C:/Users/mskim3/Desktop/식단인식_알고리즘/data/dataset.txt',encoding='UTF-8',sep='\t',index=False)

In [55]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import gluonnlp as nlp
import numpy as np
from torch.utils.data import Dataset, DataLoader
from kogpt2.pytorch_kogpt2 import get_pytorch_kogpt2_model
from gluonnlp.data import SentencepieceTokenizer
from kogpt2.utils import get_tokenizer
from tqdm import tqdm, tqdm_notebook
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule

tok_path = get_tokenizer()
gpt, vocab = get_pytorch_kogpt2_model()
tok = SentencepieceTokenizer(tok_path,  num_best=0, alpha=0)

using cached model
using cached model
using cached model


In [56]:
#device = torch.device("cuda:0")
device = torch.device("cpu")

In [157]:
dataset_train = nlp.data.TSVDataset("C:/Users/mskim3/Desktop/식단인식_알고리즘/data/fe_train.txt", field_indices=[0,1], num_discard_samples=1)
dataset_val = nlp.data.TSVDataset("C:/Users/mskim3/Desktop/식단인식_알고리즘/data/fe_val.txt", field_indices=[0,1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("C:/Users/mskim3/Desktop/식단인식_알고리즘/data/fe_test.txt", field_indices=[0,1], num_discard_samples=1)

In [170]:
class GPT2Dataset(Dataset):
    def __init__(self, dataset, vocab, in_idx, out_idx, gpt2tokenizer, max_len, pad=True):
        
        tok = gpt2tokenizer
        MASK = '<unused0>'
        F = '<unused1>'
        EF = '<unused2>'
        
        self.in_token = [vocab[['<s>'] + tok(i[in_idx]) +['</s>']] + [vocab['<sys>']] for i in dataset]
        self.pre_out_token = [[[F] + o + [EF] for o in [tok(j) for j in eval(i[out_idx])]] for i in dataset]
        self.out_token = []
        for i in self.pre_out_token:
            l = []
            if len(i) > 1:
                for j in i:
                    l = l + j
                l = l + [vocab.eos_token]
                self.out_token.append(vocab[l])
            else :
                l = l + vocab[i[0]] + [vocab[vocab.eos_token]]
                self.out_token.append(vocab[i[0]])
        
        if pad == True:
            self.pad = []
            for i in range(len(self.in_token)):
                self.pad.append([vocab[vocab.padding_token]] * (max_len - len(self.in_token[i]) - len(self.out_token[i])))
    def __getitem__(self, i):
        mask = np.array([0]*(len(self.in_token[i])-1) + [1]*len(self.out_token[i]) + [0]*len(self.pad[i]))
        return (np.array(self.in_token[i] + self.out_token[i] + self.pad[i]), mask)

    def __len__(self):
        return (len(self.pad))

In [171]:
traindata = GPT2Dataset(dataset_train,vocab, 0,1,tok,max_len=64,pad=True)
valdata = GPT2Dataset(dataset_val,vocab, 0,1,tok,max_len=64,pad=True)
testdata = GPT2Dataset(dataset_test,vocab, 0,1,tok,max_len=64,pad=True)

In [161]:
train_dataloader = torch.utils.data.DataLoader(traindata, batch_size=16, num_workers=0)
val_dataloader = torch.utils.data.DataLoader(valdata, batch_size=16, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(testdata, batch_size=16, num_workers=0)

In [162]:
class GPT2summarizer(nn.Module):
    def __init__(self, gpt):
        super(GPT2summarizer, self).__init__()
        self.gpt = gpt

    def forward(self, ids):
        out = self.gpt(ids)

        return out

In [163]:
model = GPT2summarizer(gpt).to(device)

In [164]:
max_grad_norm = .5
num_epochs = 50
warmup_ratio = 0.1
log_interval = 200
learning_rate =  .0001

In [165]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [166]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [167]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [168]:
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

#### Local small memory

In [155]:
for e in range(num_epochs):
    train_loss = 0.0
    test_loss = 0.0
    model.train()
    for batch_id, (ids, mask) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        ids = ids.long().to(device)
        mask = mask.long().to(device)
        out = model(ids)[0]
        newout = out[:,:-1,:]
        newids = ids[:,1:]
        loss = 0.0
        for i in range(mask.shape[0]):
            maskt = torch.where(mask[i,:]==1)[0]
            outt = newout[i,maskt,:]
            lab = newids[i,maskt].long()
            loss += loss_fn(outt,lab)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        
        loss_avg = (loss / torch.sum(mask)).data.cpu().numpy()
        train_loss += loss_avg
        if batch_id % log_interval == 0:
            print('Epoch: ',e+1,' Batch ID: ',batch_id,' Train Loss: ',train_loss / (batch_id+1))
    model.eval()
    for batch_id, (ids, mask) in enumerate(tqdm_notebook(val_dataloader)):
        ids = ids.long().to(device)
        mask = mask.long().to(device)
        out = model(ids)[0]
        newout = out[:,:-1:]
        newids = ids[:,1:]
        loss = 0.0
        for i in range(mask.shape[0]):
            maskt = torch.where(mask[i,:]==1)[0]
            outt = newout[i,maskt,:]
            lab = newids[i,maskt].long()
            loss += loss_fn(outt,lab)
        loss_avg = (loss / torch.sum(mask)).data.cpu().numpy()
        test_loss += loss_avg
    print('epoch: ',e+1,' Test Loss: ',test_loss / (batch_id+1))

Epoch:  1  Batch ID:  0  Train Loss:  1.3752186298370361


KeyboardInterrupt: 

#### GPU bigger memory

In [153]:
for e in range(num_epochs):
    train_loss = 0.0
    test_loss = 0.0
    model.train()
    for batch_id, (ids, mask) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        #input-output shaping
        ids = ids.long().to(device)
        mask = mask.long().to(device)
        out = model(ids)[0]
        newout = out[:,:-1:]
        newids = ids[:,1:]
        newmask = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(newmask == 1, newout, -1e-8 * torch.ones_like(newout))
        
        #loss calculation
        loss = loss_fn(mask_out.transpose(2, 1),newids)
        
        #back propagation
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        
        loss_avg = (loss / torch.sum(mask)).data.cpu().numpy()
        train_loss += loss_avg
        if batch_id % log_interval == 0:
            print('Epoch: ',e+1,' Batch ID: ',batch_id,' Train Loss: ',train_loss / (batch_id+1))
    model.eval()
    for batch_id, (ids, mask) in enumerate(tqdm_notebook(val_dataloader)):
        
        #input-output shaping
        ids = ids.long().to(device)
        mask = mask.long().to(device)
        out = model(ids)[0]
        newout = out[:,:-1:]
        newids = ids[:,1:]
        newmask = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(newmask == 1, newout, -1e-8 * torch.ones_like(newout))
        
        #Loss calculation
        loss = loss_fn(mask_out.transpose(2, 1),newids)
        loss_avg = (loss / torch.sum(mask)).data.cpu().numpy()
        test_loss += loss_avg
    print('epoch: ',e+1,' Test Loss: ',test_loss / (batch_id+1))

C:\Users\mskim3\AppData\Local\Continuum\anaconda3\lib\site-packages\transformers\optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:766.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


Epoch:  1  Batch ID:  0  Train Loss:  0.06652802973985672


KeyboardInterrupt: 